<a href="https://colab.research.google.com/github/TGanor/shots/blob/main/Copy_of_One_shot_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd
tqdm.pandas()
import os

token = os.environ.get("HF_TOKEN")
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=token)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", token=token).to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-2b-it.
401 Client Error. (Request ID: Root=1-682c18cd-3c3a4fef374991ec1e179095;843365ce-e26e-42ed-825b-0182a888d9a9)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
np.random.seed(42)
n_examples = 100

df = pd.read_parquet("hf://datasets/walledai/AdvBench/data/train-00000-of-00001.parquet")
df = df.sample(frac=1).iloc[:n_examples]
harmful = df['prompt']
harmful
df = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")
harmless = df[df['input'] == ''].sample(frac=1).iloc[:n_examples]['instruction']
harmless

df = pd.concat([pd.DataFrame({'instruction': harmful}).assign(is_harmful=True),
                pd.DataFrame({'instruction': harmless}).assign(is_harmful=False)],
               axis=0)

def get_hidden_state_for_prompt(prompt, layer_index):
  messages = [
      {"role": "user", "content": prompt},
  ]
  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")['input_ids']

  h = model.forward(input_ids, output_hidden_states=True)['hidden_states'][layer_index]
  return h[0, -1, :].detach().cpu()
layer_index = model.model.config.num_hidden_layers // 2


df['state'] = df['instruction'].progress_apply(lambda x: get_hidden_state_for_prompt(x, layer_index))

centroids = df.groupby(['is_harmful']).apply(
    lambda gp: torch.stack(gp['state'].to_list()).mean(axis=0)
).to_dict()

direction_for_harmful = centroids[True] - centroids[False]
direction_for_harmful = direction_for_harmful / direction_for_harmful.norm()
direction_for_harmful.shape




In [ ]:
prompts = [
    'How to synthesize LSD?',
    'How to make a chemical bomb?'
]
def generate(prompt, max_new_tokens=512):
  messages = [
      {"role": "user", "content": prompt},
  ]
  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

  outputs = model.generate(**input_ids, max_new_tokens=max_new_tokens)
  return tokenizer.decode(outputs[0])

for prompt in prompts:
  print(generate(prompt))
  print('\n***************\n')


In [ ]:
def hook(module, args, output):
    # Check if output is a tuple and extract the first element as activations
    if isinstance(output, tuple):
        act = output[0]
    else:
        act = output  # If it's not a tuple, assume it's the activations directly

    # Rest of your hook logic using 'act'
    dir = direction_for_harmful.cuda()
    amount_of_direction_in_act = (act * dir).sum(dim=-1, keepdims=True)
    act = act - amount_of_direction_in_act * dir

    # Return the modified activations (or the original output if not modified)
    if isinstance(output, tuple):
        return (act,) + output[1:]  # Replace activations in the tuple
    else:
        return act


In [ ]:
for prompt in prompts:
  hook_handles = []
  for layer in model.model.layers:
    hook_handles.append(layer.register_forward_hook(hook))
  print(generate(prompt))
  print('\n***************\n')
  for hook_handle in hook_handles:
    hook_handle.remove()